# Picking states and features

In [2]:
import shnitsel as st
database = st.io.read('./test_data/shnitsel/traj_I02.nc')
dataset = database['I02/0/data']
# dataset

Shnitsel tools contain a versatile selection and filtering mechanism that allows you deep control of which parts of a molecule and state space are included in an analyis. 
Since state selection and structure selection are orthogonal selection axes, we offer two separate interfaces for the filtration process:
- `shnitsel.filtration.Stateselection`, which enables the restriction of analysis to a subset of states or state combinations as well as the assignment of meta-data to those states like TeX labels, colors in plots, etc. and 
- `shnitsel.filtration.StructureSelection`, which enables the restriction of analysis to a subset of geometric features of a system. Specifically, it facilitates the restriction to specific features like `atoms`, `bonds`, `angles`, `dihedrals`, and `pyramids` or pyramidalization, which can help narrow down analysis and, therefore, speed up calculations.

Here, we discuss the methods that shnitsel tools offers to work with selecting states.
For feature/structure selection, please see the separate tutorial.

## `StateSelection`: Picking states and state combinations

The state selection is available from the module `shnitsel.filtering`, or, for more specific typing support `shnitsel.filtering.state_selection`. 
There are multiple ways to create a `StateSelection`, depending on the use case.

### Creating a specific structure selection from a dataset

If you have a dataset, you can initialize a state selection from it:

In [3]:
from shnitsel.filtering import StateSelection

# Initializes a selection of all states and state combinations
state_selection_ds = StateSelection.init_from_dataset(dataset)
state_selection_ds

StateSelection(is_directed=False, states_base=[1, 2, 3], states=[1, 2, 3], ground_state_id=np.int64(1), state_types={1: np.int8(1), 2: np.int8(1), 3: np.int8(1)}, state_names={1: np.str_('S0'), 2: np.str_('S1'), 3: np.str_('S2')}, state_charges={1: np.float32(0.0), 2: np.float32(0.0), 3: np.float32(0.0)}, state_degeneracy_group=None, degeneracy_group_states=None, state_combinations_base=[(1, 2), (1, 3), (2, 3)], state_combinations=[(1, 2), (1, 3), (2, 3)], state_combination_names={(1, 2): 'S0 - S1', (1, 3): 'S0 - S2', (2, 3): 'S1 - S2'}, state_colors=None, state_combination_colors=None)

In [4]:

# If you want to distinguish between combinations of different directions, you can set the `is_directed` parameter:
state_selection_ds_directed = StateSelection.init_from_dataset(dataset, is_directed=True)
state_selection_ds_directed

StateSelection(is_directed=True, states_base=[1, 2, 3], states=[1, 2, 3], ground_state_id=np.int64(1), state_types={1: np.int8(1), 2: np.int8(1), 3: np.int8(1)}, state_names={1: np.str_('S0'), 2: np.str_('S1'), 3: np.str_('S2')}, state_charges={1: np.float32(0.0), 2: np.float32(0.0), 3: np.float32(0.0)}, state_degeneracy_group=None, degeneracy_group_states=None, state_combinations_base=[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)], state_combinations=[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)], state_combination_names={(1, 2): 'S0 - S1', (1, 3): 'S0 - S2', (2, 1): 'S1 - S0', (2, 3): 'S1 - S2', (3, 1): 'S2 - S0', (3, 2): 'S2 - S1'}, state_colors=None, state_combination_colors=None)

If you instead want to provide a description of your states, you can initialize them from a textual description:

In [5]:
from shnitsel.filtering import StateSelection

# Initializes a selection of all states and state combinations
state_selection_descr = StateSelection.init_from_descriptor("1,2,3,4, 3->1, 2->1, 3<>1")
state_selection_descr

StateSelection(is_directed=True, states_base=[1, 2, 3, 4], states=[1, 2, 3, 4], ground_state_id=np.int64(1), state_types=None, state_names=None, state_charges=None, state_degeneracy_group=None, degeneracy_group_states=None, state_combinations_base=[(3, 1), (1, 3), (2, 1)], state_combinations=[(3, 1), (1, 3), (2, 1)], state_combination_names=None, state_colors=None, state_combination_colors=None)

## Selecting further within a selection

`StateSelection` objects support restricting the current selection further either specifically for states or transitions via the `.select_states()` and `.select_state_combinations()` methods or both at the same time via the `.select()` method. 

The methods support either lists of respective ids or of textual representations as above. 
If your selection has been initialized from a dataset that has multiplicity or state name information or if you have configured your selection manually to include those (by using methods like `.set_state_names()`, `.set_state_types()` or `.set_state_charges()`) you can even match multiplicity labels, i.e. 
- `'T'` would match all triplet states
- `'3 -> S'` would match all transitions from state 3 to a singlet state.
  - Note: if any such directed selector is present, the selection will be converted into a directed selection.
- `'S3 -> 1` would match the transition from a state named `S3` to the state with id `1`.
Note, however, that matching will fail with an exception if the necessary information is missing.

In [6]:
state_selection_ds.select_states("S, 1->S, S1<>S0")
state_selection_ds.select_state_combinations("S, 1->S, S1<>S0")
state_selection_ds.select("S, 1->S, S1<>S0")

StateSelection(is_directed=True, states_base=[1, 2, 3], states=[1, 2, 3], ground_state_id=np.int64(1), state_types={1: np.int8(1), 2: np.int8(1), 3: np.int8(1)}, state_names={1: np.str_('S0'), 2: np.str_('S1'), 3: np.str_('S2')}, state_charges={1: np.float32(0.0), 2: np.float32(0.0), 3: np.float32(0.0)}, state_degeneracy_group=None, degeneracy_group_states=None, state_combinations_base=[(1, 2), (1, 3), (2, 3), (2, 1), (3, 1), (3, 2)], state_combinations=[(1, 2), (1, 3), (2, 1)], state_combination_names={(1, 2): 'S0 - S1', (1, 3): 'S0 - S2', (2, 3): 'S1 - S2', (2, 1): 'S1 - S0', (3, 1): 'S2 - S0', (3, 2): 'S2 - S1'}, state_colors=None, state_combination_colors=None)

Be careful that the results of the three above calls are different. 
- `.select_states()` will only change the states selection, not the transition selection. 
- `.select_state_combinations()` will impact only the transition selection, not the state selection
- `.select()` will impact both selections

A common mistake is to call `.select()` with a selector for only combinations:

In [7]:
state_selection_ds.select("1->2").states

[]

Note that the selection no longer includes any states because there is no explicit statement describing the states you wish to include. 
You can fix this either by including a statement like `'S'` to include all singlet states, by setting `states_from_sc=True` to make the `.select()` statement extract state information from state combinations or by calling `.select_state_combinations()` directly, e.g.:

In [8]:
state_selection_ds.select(["S", "1->2"])
state_selection_ds.select("1->2", states_from_sc=True)
state_selection_ds.select_state_combinations("1->2")

StateSelection(is_directed=True, states_base=[1, 2, 3], states=[1, 2, 3], ground_state_id=np.int64(1), state_types={1: np.int8(1), 2: np.int8(1), 3: np.int8(1)}, state_names={1: np.str_('S0'), 2: np.str_('S1'), 3: np.str_('S2')}, state_charges={1: np.float32(0.0), 2: np.float32(0.0), 3: np.float32(0.0)}, state_degeneracy_group=None, degeneracy_group_states=None, state_combinations_base=[(1, 2), (1, 3), (2, 3), (2, 1), (3, 1), (3, 2)], state_combinations=[(1, 2)], state_combination_names={(1, 2): 'S0 - S1', (1, 3): 'S0 - S2', (2, 3): 'S1 - S2', (2, 1): 'S1 - S0', (3, 1): 'S2 - S0', (3, 2): 'S2 - S1'}, state_colors=None, state_combination_colors=None)

Additional helper methods are defined, e.g. for selecting only singlets or only triplets:

In [9]:
state_selection_ds_directed.singlets_only()
state_selection_ds_directed.triplets_only()

StateSelection(is_directed=True, states_base=[1, 2, 3], states=[], ground_state_id=np.int64(1), state_types={1: np.int8(1), 2: np.int8(1), 3: np.int8(1)}, state_names={1: np.str_('S0'), 2: np.str_('S1'), 3: np.str_('S2')}, state_charges={1: np.float32(0.0), 2: np.float32(0.0), 3: np.float32(0.0)}, state_degeneracy_group=None, degeneracy_group_states=None, state_combinations_base=[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)], state_combinations=[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)], state_combination_names={(1, 2): 'S0 - S1', (1, 3): 'S0 - S2', (2, 1): 'S1 - S0', (2, 3): 'S1 - S2', (3, 1): 'S2 - S0', (3, 2): 'S2 - S1'}, state_colors=None, state_combination_colors=None)

There are also support functions to only consider transitions between different multiplicities of any kind:

In [10]:
state_selection_ds.different_multiplicity_transitions()

StateSelection(is_directed=False, states_base=[1, 2, 3], states=[1, 2, 3], ground_state_id=np.int64(1), state_types={1: np.int8(1), 2: np.int8(1), 3: np.int8(1)}, state_names={1: np.str_('S0'), 2: np.str_('S1'), 3: np.str_('S2')}, state_charges={1: np.float32(0.0), 2: np.float32(0.0), 3: np.float32(0.0)}, state_degeneracy_group=None, degeneracy_group_states=None, state_combinations_base=[(1, 2), (1, 3), (2, 3)], state_combinations=[], state_combination_names={(1, 2): 'S0 - S1', (1, 3): 'S0 - S2', (2, 3): 'S1 - S2'}, state_colors=None, state_combination_colors=None)

Additionally, if degeneracy data is available on the dataset, you can select to only include transitions between different degeneracy groups, i.e. only between states that are different without an external magnetic field.

In [11]:
state_selection_ds.non_degenerate()

StateSelection(is_directed=False, states_base=[1, 2, 3], states=[1, 2, 3], ground_state_id=np.int64(1), state_types={1: np.int8(1), 2: np.int8(1), 3: np.int8(1)}, state_names={1: np.str_('S0'), 2: np.str_('S1'), 3: np.str_('S2')}, state_charges={1: np.float32(0.0), 2: np.float32(0.0), 3: np.float32(0.0)}, state_degeneracy_group=None, degeneracy_group_states=None, state_combinations_base=[(1, 2), (1, 3), (2, 3)], state_combinations=[(1, 2), (1, 3), (2, 3)], state_combination_names={(1, 2): 'S0 - S1', (1, 3): 'S0 - S2', (2, 3): 'S1 - S2'}, state_colors=None, state_combination_colors=None)